<a href="https://colab.research.google.com/github/mkbahk/DeepLearningwithKeras/blob/master/__Useful_Code_Sample_mkbahk_20201021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EarlyStopping

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=15, mode='auto', monitor='val_acc')
hist = model.fit_generator(train_generator,
                           steps_per_epoch=train_generator.samples // batch_size,
                           validation_data = validation_generator,
                           epochs=100,
                           callbacks=[early_stopping]
                           )


훈련 및 평가 그래프 그리기

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig, loss_ax =  plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'g', label='validation loss')
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'r', label='validation acc')

plt.legend(['train acc', 'validation acc'])
plt.show()

Tensorboard


In [ ]:
import numpy as np
import tensorflow as tf
import os as os
from tensorflow import keras
from datetime import datetime

!rm -rf ./logs/ #to delete previous runs
%load_ext tensorboard
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
#%tensorboard --logdir logs



#예측과 검증이 끝난 후에 아래 코드 삽입
%tensorboard --logdir logs



모델 저장하기

In [ ]:
model.save('cats_and_dogs_small_1.h5')

훈련 시간 측정


In [ ]:
import time
import os

start = time.time() # 시작 시간 저장


print("실행시간 :", time.time() - start,"(초)")  # 현재시각 - 시작시간 = 실행 시간


실행시간 : 2.288818359375e-05 (초)


TPU 설정

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

from tensorflow import keras
import time
import os

print("Tensorflow version " + tf.__version__)
print(tf.config.list_physical_devices("CPU"))
print(tf.config.list_physical_devices("GPU"))


resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://'+os.environ['COLAB_TPU_ADDR'])
#Connect to the TPU handle and initialise it
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)


# Create the model using the IPU-specific Sequential class instead of the
# standard tf.keras.Sequential class
def create_model():
    model = tf.keras.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(10, activation='softmax')])
    
    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                  optimizer = tf.keras.optimizers.Adam(), 
                  experimental_steps_per_execution = 50, 
                  metrics=['sparse_categorical_accuracy'])
    return model



def main():
      # Get the training dataset.
      print("==============================Getting Training DataSet==============================\n\n")
      ds1 = create_train_dataset()
      print("==============================Getting Test DataSet==============================\n\n")
      ds2 = create_test_dataset()

      with strategy.scope():   
        # Create an instance of the model.
        print("==============================Building Model & Compile ==============================\n\n")
        model = create_model()
      #end of with

GPU 분산전략

In [ ]:
import tensorflow as tf
from tensorflow import keras
import time, os

# Create a MirroredStrategy.
print(tf.config.list_physical_devices("CPU"))
print(tf.config.list_physical_devices("GPU"))
gpus = tf.config.experimental.list_logical_devices("GPU")

if len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('\n\nRunning on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('\n\nRunning on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print("'\n\nRunning on CPU '")
#end of if
 
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)

CPU 분산전략

In [ ]:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print("'\n\nRunning on CPU '")
#end of if
 
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()


IPU 분산전략

In [ ]:
from tensorflow.python import ipu
print(tf.config.list_physical_devices("IPU"), "\n\n")

# Configure the IPU system
cfg = ipu.utils.create_ipu_config()
cfg = ipu.utils.auto_select_ipus(cfg, 2)
ipu.utils.configure_ipu_system(cfg)



strategy = ipu.ipu_strategy.IPUStrategy()
   with strategy.scope():
      model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = tf.keras.optimizers.Adam(),  metrics=['sparse_categorical_accuracy'])
      model.fit(ds1, steps_per_epoch=2000, epochs=50)

      print("\n\n==============================Checking the result==============================\n\n")
      (loss, accuracy) = model.evaluate(ds2, steps=1000)
      print("Validation loss: {}".format(loss))
      print("Validation accuracy: {}%".format(100.0 * accuracy))
      print("\n\n==============================Job Done by Graphcore IPU with 16 Sockets, 19,456 Cores!!!==============================")
   #end of with:
#end of def

이미지 보여주기

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

Confusion Matrix

In [ ]:
predictions = model.predict(x=scaled_test_samples, batch_size=10, verbose=0) 
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0) 

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(test_labels, rounded_predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['no_side_effects','had_side_effects']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')